In [5]:
import pandas as pd
import numpy as np
import glob
import scipy.stats as ss

In [1]:
!pip install --upgrade pandas

In [2]:
!pip install h2o

In [23]:
filtered.loc[:,'GRDE_CODE_FINAL'] = filtered.GRDE_CODE_FINAL.astype(float)

In [37]:
### Vamos a cambiar la calificacion del curso por 0 y 1. 1 Si es superior a 3 y 0 si es inferior
data.loc[:,'GRDE_CODE_FINAL'] = data['GRDE_CODE_FINAL'].apply(lambda x: 1 if x>=3 else 0)

In [41]:
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
data = pd.read_csv('final_data.csv')

In [58]:
train, valid = train_test_split(data, test_size=0.3, train_size=0.7, shuffle=True, stratify=data['GRDE_CODE_FINAL'])

In [59]:
train.to_csv('train.csv', header=False)
valid.to_csv('valid.csv', header=False)

In [43]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 20 mins
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_migeruj_r574ww
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.629 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


In [60]:
train = h2o.H2OFrame(train)
valid = h2o.H2OFrame(valid)

# Identify predictors and response
x = [col for col in train.columns if col !='GRDE_CODE_FINAL']
y = "GRDE_CODE_FINAL"

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
valid[y] = valid[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train, validation_frame = valid)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu11.43+55-CA (build 11.0.9.1+1-LTS, mixed mode)
  Starting server from C:\Users\migeruj\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\migeruj\AppData\Local\Temp\tmpf5tuc_3q
  JVM stdout: C:\Users\migeruj\AppData\Local\Temp\tmpf5tuc_3q\h2o_migeruj_started_from_python.out
  JVM stderr: C:\Users\migeruj\AppData\Local\Temp\tmpf5tuc_3q\h2o_migeruj_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 9 days
H2O_cluster_name:,H2O_from_python_migeruj_r574ww
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.969 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |
22:36:25.970: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
22:36:25.984: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20210628_223625,0.939511,0.11054,0.992418,0.190888,0.164722,0.0271334
StackedEnsemble_AllModels_AutoML_20210628_223625,0.939212,0.110609,0.992441,0.190073,0.164753,0.0271436
GBM_grid__1_AutoML_20210628_223625_model_4,0.938194,0.112697,0.992244,0.177901,0.165321,0.027331
DRF_1_AutoML_20210628_223625,0.925367,0.117928,0.990039,0.188652,0.167036,0.027901
GBM_5_AutoML_20210628_223625,0.925269,0.117712,0.990359,0.18973,0.167596,0.0280883
GBM_grid__1_AutoML_20210628_223625_model_2,0.92431,0.127475,0.990263,0.187287,0.170147,0.0289499
GBM_4_AutoML_20210628_223625,0.921302,0.11816,0.989424,0.190335,0.167476,0.0280484
GBM_2_AutoML_20210628_223625,0.917778,0.119282,0.989088,0.186087,0.167832,0.0281677
GBM_3_AutoML_20210628_223625,0.917223,0.119492,0.988805,0.189355,0.167861,0.0281772
GBM_1_AutoML_20210628_223625,0.911611,0.121567,0.988191,0.191554,0.168909,0.0285301


In [11]:
aml.leader.params.keys()

dict_keys(['model_id', 'training_frame', 'response_column', 'validation_frame', 'blending_frame', 'base_models', 'metalearner_algorithm', 'metalearner_nfolds', 'metalearner_fold_assignment', 'metalearner_fold_column', 'metalearner_params', 'metalearner_transform', 'max_runtime_secs', 'weights_column', 'offset_column', 'seed', 'score_training_samples', 'keep_levelone_frame', 'export_checkpoints_dir', 'auc_type'])

In [12]:
aml.leader.params['metalearner_algorithm']

{'default': 'AUTO', 'actual': 'glm', 'input': 'AUTO'}

In [13]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])

In [14]:
m = h2o.get_model(model_ids[0])

In [48]:
m.confusion_matrix()


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7166131850946544: 


,,0,1,Error,Rate
0,0,693.0,78.0,0.1012,(78.0/771.0)
1,1,24.0,9177.0,0.0026,(24.0/9201.0)
2,Total,717.0,9255.0,0.0102,(102.0/9972.0)


In [17]:
# retrieve the model performance on train
perf = m.model_performance(train)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.012916223546228577
RMSE: 0.11364956465481325
LogLoss: 0.05526500730153155
Null degrees of freedom: 58157
Residual degrees of freedom: 58154
Null deviance: 32353.776205407445
Residual deviance: 6428.204589284944
AIC: 6436.204589284944
AUC: 0.9947692596449296
AUCPR: 0.9995098614477785
Gini: 0.9895385192898591

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7554052479191383: 


,,0,1,Error,Rate
0,0,4178.0,460.0,0.0992,(460.0/4638.0)
1,1,251.0,53269.0,0.0047,(251.0/53520.0)
2,Total,4429.0,53729.0,0.0122,(711.0/58158.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.755405,0.993371,210.0
1,max f2,0.560289,0.996110,261.0
2,max f0point5,0.840085,0.993016,182.0
3,max accuracy,0.755405,0.987775,210.0
4,max precision,0.999598,1.000000,0.0
5,max recall,0.036203,1.000000,373.0
6,max specificity,0.999598,1.000000,0.0
7,max absolute_mcc,0.755405,0.915244,210.0
8,max min_per_class_accuracy,0.901812,0.965071,150.0
9,max mean_per_class_accuracy,0.877277,0.965911,165.0



Gains/Lift Table: Avg response rate: 92,03 %, avg score: 91,54 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010007,0.998545,1.086659,1.086659,1.000000,0.999076,1.000000,0.999076,0.010874,0.010874,8.665919,8.665919,0.010874
1,2,0.020014,0.998100,1.086659,1.086659,1.000000,0.998305,1.000000,0.998691,0.010874,0.021749,8.665919,8.665919,0.021749
2,3,0.030004,0.997792,1.086659,1.086659,1.000000,0.997942,1.000000,0.998441,0.010856,0.032605,8.665919,8.665919,0.032605
3,4,0.040012,0.997526,1.086659,1.086659,1.000000,0.997654,1.000000,0.998245,0.010874,0.043479,8.665919,8.665919,0.043479
4,5,0.050002,0.997285,1.086659,1.086659,1.000000,0.997403,1.000000,0.998076,0.010856,0.054335,8.665919,8.665919,0.054335
5,6,0.100003,0.996207,1.086659,1.086659,1.000000,0.996736,1.000000,0.997406,0.054335,0.108670,8.665919,8.665919,0.108670
6,7,0.150005,0.995176,1.086659,1.086659,1.000000,0.995702,1.000000,0.996838,0.054335,0.163004,8.665919,8.665919,0.163004
7,8,0.200007,0.994148,1.086659,1.086659,1.000000,0.994667,1.000000,0.996295,0.054335,0.217339,8.665919,8.665919,0.217339
8,9,0.300010,0.991941,1.086659,1.086659,1.000000,0.993084,1.000000,0.995225,0.108670,0.326009,8.665919,8.665919,0.326009
9,10,0.399997,0.989307,1.086659,1.086659,1.000000,0.990659,1.000000,0.994084,0.108651,0.434660,8.665919,8.665919,0.434660


In [18]:
# retrieve the model performance on train
perf = m.model_performance(valid)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.026086288008978522
RMSE: 0.16151250109195425
LogLoss: 0.1070821736642612
Null degrees of freedom: 24925
Residual degrees of freedom: 24922
Null deviance: 13867.491633341506
Residual deviance: 5338.26052151075
AIC: 5346.26052151075
AUC: 0.940891348176249
AUCPR: 0.9925843498768238
Gini: 0.881782696352498

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.29213575808734477: 


,,0,1,Error,Rate
0,0,1265.0,723.0,0.3637,(723.0/1988.0)
1,1,27.0,22911.0,0.0012,(27.0/22938.0)
2,Total,1292.0,23634.0,0.0301,(750.0/24926.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.292136,0.983896,327.0
1,max f2,0.131541,0.993001,349.0
2,max f0point5,0.794126,0.977943,224.0
3,max accuracy,0.355694,0.969911,319.0
4,max precision,0.996981,0.998666,7.0
5,max recall,0.001972,1.000000,398.0
6,max specificity,0.999589,0.999497,0.0
7,max absolute_mcc,0.292136,0.776169,327.0
8,max min_per_class_accuracy,0.951393,0.862676,114.0
9,max mean_per_class_accuracy,0.929938,0.874392,141.0



Gains/Lift Table: Avg response rate: 92,02 %, avg score: 91,94 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010030,0.998429,1.082322,1.082322,0.996000,0.998935,0.996000,0.998935,0.010855,0.010855,8.232174,8.232174,0.010352
1,2,0.020019,0.998020,1.086668,1.084491,1.000000,0.998213,0.997996,0.998575,0.010855,0.021711,8.666841,8.449072,0.021208
2,3,0.030049,0.997701,1.082322,1.083767,0.996000,0.997870,0.997330,0.998339,0.010855,0.032566,8.232174,8.376676,0.031560
3,4,0.040039,0.997404,1.086668,1.084491,1.000000,0.997555,0.997996,0.998143,0.010855,0.043421,8.666841,8.449072,0.042415
4,5,0.050189,0.997110,1.086668,1.084931,1.000000,0.997249,0.998401,0.997963,0.011030,0.054451,8.666841,8.493113,0.053445
5,6,0.100016,0.995951,1.083169,1.084053,0.996779,0.996518,0.997593,0.997243,0.053972,0.108423,8.316867,8.405308,0.105405
6,7,0.150004,0.994912,1.085796,1.084634,0.999197,0.995429,0.998128,0.996638,0.054277,0.162699,8.579628,8.463400,0.159178
7,8,0.200193,0.993795,1.078851,1.083184,0.992806,0.994364,0.996794,0.996068,0.054146,0.216845,7.885065,8.318410,0.208797
8,9,0.300008,0.991436,1.080554,1.082309,0.994373,0.992664,0.995988,0.994935,0.107856,0.324701,8.055372,8.230895,0.309611
9,10,0.400024,0.988540,1.077515,1.081110,0.991576,0.990036,0.994885,0.993710,0.107769,0.432470,7.751477,8.111028,0.406816


In [44]:
latest_model = h2o.load_model(path='./StackedEnsemble_BestOfFamily_AutoML_20210628_223625')

In [61]:
latest_model.predict(valid)

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1
1,0.0426635,0.957336
1,0.00288775,0.997112
1,0.0133689,0.986631
1,0.00544803,0.994552
1,0.0122344,0.987766
1,0.0124,0.9876
1,0.00865593,0.991344
1,0.0139723,0.986028
1,0.0100992,0.989901
1,0.014256,0.985744
